In [1]:
!git clone https://github.com/d-atallah/implicit_gender_bias.git

fatal: destination path 'implicit_gender_bias' already exists and is not an empty directory.


In [2]:
from implicit_gender_bias import config as cf
import pandas as pd
import joblib
import gensim

In [3]:
filepath = cf.filepath()
files = ['annotations']
df_dict = cf.extract_dfs(filepath, files)
cf.load_dfs(filepath, df_dict)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
